In [1]:
import numpy as np
seed = 123
np.random.seed(seed)

import os
import pandas as pd
from scipy import sparse
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils 
from keras.optimizers import Adagrad
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from rpy2.robjects.packages import importr

Using TensorFlow backend.


ModuleNotFoundError: No module named 'rpy2'

In [1]:
from rpy2.robjects.packages import importr

OSError: cannot load library 'C:\Users\RISHABH\Anaconda3\lib\R\bin\x64\R.dll': error 0x7e

In [2]:
RAW_DATA_DIR='C:\\Users\\RISHABH\\Documents\\input'

## Load train for all individual models

In [159]:
train_xgb_events=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_train_xgbevents.csv')) 
train_keras1=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_keras_cv10.csv')) 
train_xgb_noevents=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_train_xgbnoevents.csv'))
train_keras2_events=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_train_keras2_events.csv')) 
train_keras2_noevents=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_train_keras2_noevents.csv')) 
train_keras3=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_train_keras3.csv'))
train_keras4=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_train_cv10_keras4.csv')) 
events = pd.read_csv(os.path.join(RAW_DATA_DIR, 'events.csv'),
                     parse_dates=['timestamp'],
                     infer_datetime_format=True,
                     )
gatrain = pd.read_csv(os.path.join(RAW_DATA_DIR,'gender_age_train.csv'),
                      ) 

## Load test for all individual models

In [176]:
test_xgb_events=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_test_xgbevents.csv')) 
test_keras1=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_keras_test_bag10.csv')) 
test_xgb_noevents=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_test_xgbnoevents.csv'))
test_keras2_events=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_test_keras2_events.csv')) 
test_keras2_noevents=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_test_keras2_noevents.csv')) 
test_keras3=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_test_keras3.csv'))
test_keras4=pd.read_csv(os.path.join(RAW_DATA_DIR,'pred_test_cv10_keras4.csv')) 
events = pd.read_csv(os.path.join(RAW_DATA_DIR, 'events.csv'),
                     parse_dates=['timestamp'],
                     infer_datetime_format=True,
                     )
gatest = pd.read_csv(os.path.join(RAW_DATA_DIR,'gender_age_test.csv'),
                      ) 

## Adding Has events to models

In [162]:
s=events.device_id.unique() 
#creating featura has events for train and test
train_keras1['has_events']=train_keras1.device_id.apply(lambda x:1 if x in s else 0)  
train_keras3['has_events']=train_keras3.device_id.apply(lambda x:1 if x in s else 0) 
train_keras4['has_events']=train_keras4.device_id.apply(lambda x:1 if x in s else 0) 
gatrain['has_events']=gatrain.device_id.apply(lambda x:1 if x in s else 0)

In [145]:
train_keras1_events=train_keras1[train_keras1['has_events']==1] 
train_keras3_events=train_keras3[train_keras3['has_events']==1] 
train_keras4_events=train_keras4[train_keras4['has_events']==1] 
gatrain=gatrain[gatrain['has_events']==1]

In [163]:
test_keras1['has_events']=test_keras1.device_id.apply(lambda x:1 if x in s else 0)  
test_keras3['has_events']=test_keras3.device_id.apply(lambda x:1 if x in s else 0) 
test_keras4['has_events']=test_keras4.device_id.apply(lambda x:1 if x in s else 0) 


In [147]:
test_keras1_events=test_keras1[test_keras1['has_events']==1] 
test_keras3_events=test_keras3[test_keras3['has_events']==1] 
test_keras4_events=test_keras4[test_keras4['has_events']==1] 

In [148]:
ytrain = gatrain['group']
label_group = LabelEncoder()
ytrain = label_group.fit_transform(ytrain)

In [149]:
train_keras1_events=train_keras1_events.drop(['has_events',],axis=1) 
train_keras3_events=train_keras3_events.drop(['has_events'],axis=1) 
train_keras4_events=train_keras4_events.drop(['has_events'],axis=1) 


In [150]:
train_keras1_events=train_keras1_events.drop(['Unnamed: 0',],axis=1)

In [151]:
test_keras1_events=test_keras1_events.drop(['has_events','Unnamed: 0'],axis=1) 
test_keras3_events=test_keras3_events.drop(['has_events'],axis=1) 
test_keras4_events=test_keras4_events.drop(['has_events'],axis=1) 

In [152]:
train_xgb_events=train_xgb_events.to_numpy() 
train_keras1_events=train_keras1_events.to_numpy() 
train_keras2_events=train_keras2_events.to_numpy() 
train_keras3_events=train_keras3_events.to_numpy() 
train_keras4_events=train_keras4_events.to_numpy()

In [153]:
test_xgb_events=test_xgb_events.to_numpy() 
test_keras1_events=test_keras1_events.to_numpy() 
test_keras2_events=test_keras2_events.to_numpy() 
test_keras3_events=test_keras3_events.to_numpy() 
test_keras4_events=test_keras4_events.to_numpy()

## Stacking trained predictions for events data

In [154]:
meta_train=np.hstack((train_xgb_events[:,1:],train_keras1_events[:,1:],train_keras2_events[:,1:],train_keras3_events[:,1:],train_keras4_events[:,1:]))

## Stacking test predictions for events data

In [155]:
test=np.hstack((test_xgb_events[:,1:],test_keras1_events[:,1:],test_keras2_events[:,1:],test_keras3_events[:,1:],test_keras4_events[:,1:]))

r function for python

## Using logistic regression as metalearner for events data

In [158]:
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression(C=0.06, multi_class='multinomial', solver='lbfgs').fit(meta_train, ytrain) 
p_group=np.zeros((test.shape[0],12)) 
p_group+=clf.predict_proba(test)

In [164]:
train_keras1_noevents=train_keras1[train_keras1['has_events']==0] 
train_keras3_noevents=train_keras3[train_keras3['has_events']==0] 
train_keras4_noevents=train_keras4[train_keras4['has_events']==0] 
gatrain=gatrain[gatrain['has_events']==0]

In [165]:
test_keras1_noevents=test_keras1[test_keras1['has_events']==0] 
test_keras3_noevents=test_keras3[test_keras3['has_events']==0] 
test_keras4_noevents=test_keras4[test_keras4['has_events']==0]

In [166]:
ytrain = gatrain['group']
label_group = LabelEncoder()
ytrain = label_group.fit_transform(ytrain)

In [167]:
train_keras1_noevents=train_keras1_noevents.drop(['has_events','Unnamed: 0'],axis=1) 
train_keras3_noevents=train_keras3_noevents.drop(['has_events'],axis=1) 
train_keras4_noevents=train_keras4_noevents.drop(['has_events'],axis=1) 

In [168]:
test_keras1_noevents=test_keras1_noevents.drop(['has_events','Unnamed: 0'],axis=1) 
test_keras3_noevents=test_keras3_noevents.drop(['has_events'],axis=1) 
test_keras4_noevents=test_keras4_noevents.drop(['has_events'],axis=1) 

In [169]:
test_xgb_noevents=test_xgb_noevents.drop(['has_events','Unnamed: 0'],axis=1)

In [121]:
train_xgb_noevents.head()

,device_id,0,1,2,3,4,5,6,7,8,9,10,11
0,-8076087639492063270,0.040373,0.063003,0.046480,0.060038,0.061173,0.041472,0.074010,0.174139,0.098744,0.113983,0.124493,0.102093
1,-2897161552818060146,0.040314,0.059087,0.044647,0.053344,0.055560,0.040670,0.081050,0.178463,0.103223,0.110434,0.123781,0.109427
2,-4938849341048082022,0.061272,0.053939,0.041866,0.068234,0.083065,0.066693,0.087003,0.115051,0.067988,0.099514,0.129951,0.125425
3,245133531816851882,0.075852,0.065344,0.048002,0.063929,0.064828,0.044747,0.113037,0.146881,0.082414,0.098407,0.107172,0.089388
4,-1297074871525174196,0.122233,0.070525,0.051627,0.066755,0.073594,0.045431,0.139238,0.121128,0.068153,0.078752,0.093067,0.069497


In [170]:
train_xgb_noevents=train_xgb_noevents.to_numpy() 
train_keras1_noevents=train_keras1_noevents.to_numpy() 
train_keras2_noevents=train_keras2_noevents.to_numpy() 
train_keras3_noevents=train_keras3_noevents.to_numpy() 
train_keras4_noevents=train_keras4_noevents.to_numpy()

In [171]:
test_xgb_noevents=test_xgb_noevents.to_numpy() 
test_keras1_noevents=test_keras1_noevents.to_numpy() 
test_keras2_noevents=test_keras2_noevents.to_numpy() 
test_keras3_noevents=test_keras3_noevents.to_numpy() 
test_keras4_noevents=test_keras4_noevents.to_numpy()

## Stacking trained predictions for noevents data

In [172]:
meta_train=np.hstack((train_xgb_noevents[:,1:],train_keras1_noevents[:,1:],train_keras2_noevents[:,1:],train_keras3_noevents[:,1:],train_keras4_noevents[:,1:])) 
test=np.hstack((test_xgb_noevents[:,1:],test_keras1_noevents[:,1:],test_keras2_noevents[:,1:],test_keras3_noevents[:,1:],test_keras4_noevents[:,1:]))

In [125]:
meta_train.shape

(51336, 60)

## Using logistic regression as metalearner for noevents data

In [173]:
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression(C=0.06, multi_class='multinomial', solver='lbfgs').fit(meta_train, ytrain) 
p_group=np.zeros((test.shape[0],12)) 
p_group+=clf.predict_proba(test)

In [128]:
dtest=xgb.DMatrix(test)
p_group=np.zeros((test.shape[0],12)) 
p_group+=xgb_group.predict(dtest)

In [174]:
p_group_noevents=p_group

In [177]:
final_pred_noevents=pd.DataFrame(p_group_noevents,index=test_xgb_noevents.device_id,columns=list(label_group.classes_)) 
final_pred_noevents=final_pred_noevents.reset_index()  

final_pred_events=pd.DataFrame(p_group_events,index=test_xgb_events.device_id,columns=list(label_group.classes_)) 
final_pred_events=final_pred_events.reset_index() 



## Combining final prediction

In [178]:
gatest = pd.read_csv(os.path.join(RAW_DATA_DIR,'gender_age_test.csv'),
                     )
pred_test=final_pred_noevents.append(final_pred_events,ignore_index=True) 
gatest=gatest.merge(pred_test,on='device_id') 
gatest.to_csv('final_sub.csv',index=False)

In [141]:
gatest

,device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
0,1002079943728939269,0.000572,0.000507,0.001270,0.002703,0.013747,0.009861,0.000594,0.006339,0.013291,0.099033,0.140666,0.711418
1,-1547860181818787117,0.001123,0.005265,0.004131,0.014236,0.030367,0.026144,0.001583,0.035844,0.079710,0.266303,0.410740,0.124554
2,7374582448058474277,0.004467,0.018456,0.035311,0.134761,0.124947,0.070306,0.014288,0.027305,0.041761,0.178684,0.251354,0.098361
3,-6220210354783429585,0.002962,0.007695,0.002018,0.005748,0.032360,0.054211,0.014240,0.223050,0.143202,0.167419,0.167737,0.179358
4,-5893464122623104785,0.027091,0.070482,0.026176,0.076757,0.028917,0.027073,0.064716,0.214713,0.130337,0.104152,0.126223,0.103363
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112066,4280900819321920929,0.032084,0.097028,0.024081,0.046159,0.085447,0.039708,0.067062,0.112235,0.051735,0.109589,0.088480,0.246392
112067,818534825520551359,0.066514,0.057441,0.035985,0.069576,0.088986,0.063844,0.099915,0.107768,0.077571,0.096744,0.123366,0.112289
112068,-8956851351560395765,0.035106,0.036281,0.020812,0.064533,0.017959,0.007202,0.069170,0.297113,0.125359,0.232045,0.051672,0.042747
112069,6097318236795836256,0.035106,0.036281,0.020812,0.064533,0.017959,0.007202,0.069170,0.297113,0.125359,0.232045,0.051672,0.042747


In [ ]:
log_loss(pred_t)